In [1]:
import os
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from caserec.utils.process_data import ReadFile, ReadDataframe
from caserec.recommenders.item_recommendation.most_popular import MostPopular
from IPython.display import clear_output

In [2]:
train_file = './../../../../Datasets/Amazon/InstantVideo/ratings.csv'
# train_file = './../../../../Datasets/MovieLens/1M_raw/ratings.dat'
# dict_ratings_from_file = ReadFile(train_file, sep='\t').read() 
df_feedback = pd.read_csv(train_file, sep='\t', header=0, names = ['user', 'item', 'feedback_value', 'timestamp'])
df_feedback.head()

,user,item,feedback_value,timestamp
0,45550,0,5.0,1202256000
1,367442,0,5.0,1198195200
2,212056,1,4.0,1215388800
3,354802,1,4.0,1185840000
4,344843,1,2.0,1281052800


In [4]:
rank_length = 10
random_state = 31415

In [5]:
df_train, df_test = train_test_split(df_feedback, test_size=0.3, random_state=random_state)
print ("Train size: {} \nTest size: {}".format(df_train.shape[0], df_test.shape[0]))

Train size: 408753 
Test size: 175180


In [6]:
model = MostPopular(train_file=df_train, test_file=df_test, rank_length=30, as_binary=True)

In [ ]:
%%time
model.compute(verbose=True)

In [ ]:
model.ranking[:5]

In [7]:
df_item_count = df_feedback[['user', 'item']].groupby(by=['item']).count()#.sort_values(['user'], ascending = False)
df_item_count.columns = ['count']

In [8]:
df_feedback.head()

,user,item,feedback_value,timestamp
0,45550,0,5.0,1202256000
1,367442,0,5.0,1198195200
2,212056,1,4.0,1215388800
3,354802,1,4.0,1185840000
4,344843,1,2.0,1281052800


In [115]:
list_users = list(np.sort(df_feedback['user'].unique()))
list_items = list(np.sort(df_feedback['item'].unique()))            
list_users[:5]

[0, 1, 2, 3, 4]

In [39]:
df_feedback.set_index(['user'], inplace=True)
df_feedback.head()

,item,feedback_value,timestamp
user,,,
45550,0,5.0,1202256000
367442,0,5.0,1198195200
212056,1,4.0,1215388800
354802,1,4.0,1185840000
344843,1,2.0,1281052800


In [108]:
%%time
dict_feedback = {}
items_seen_by_user = {}
for user_id in list_users[:100000]:
    df_user_temp = df_feedback.loc[user_id].copy()    
    items = df_user_temp['item']
    if len(df_user_temp.shape) == 1: # Only 1 row was found
        values = df_user_temp['feedback_value'] if False else 1
        items_seen_by_user[user_id] = [items]
        dict_feedback[user_id] = {items: values}
    else:
        values = df_user_temp['feedback_value'] if False else np.repeat(1, df_user_temp['feedback_value'].shape[0]) 
        items_seen_by_user[user_id] = list(items)
        dict_feedback[user_id] = dict(zip(items, values))    
    

# del df_user_temp

Wall time: 1min 28s


In [116]:
df_feedback.reset_index(drop = False, inplace = True)
df_feedback.head()

,index,item,user,feedback_value,timestamp
0,0,0,45550,5.0,1202256000
1,1,0,367442,5.0,1198195200
2,2,1,212056,4.0,1215388800
3,3,1,354802,4.0,1185840000
4,4,1,344843,2.0,1281052800


In [120]:
df_feedback.set_index(['item'], inplace = True)
df_feedback.head()

,user,feedback_value,timestamp
item,,,
0,45550,5.0,1202256000
0,367442,5.0,1198195200
1,212056,4.0,1215388800
1,354802,4.0,1185840000
1,344843,2.0,1281052800


In [136]:
%%time
users_viewed_item = {}  
for item_id in list_items:
    users = df_feedback.loc[item_id]['user'].copy()
    users_viewed_item[item_id] = [users] if len(users.shape) == 0 else list(users)
    
#     if len(users.shape) == 0:
#         users_viewed_item[item_id] = [users]
#     else:
#         users_viewed_item[item_id] = list(users)
    

Wall time: 5.83 s


In [134]:
len(users.shape)

0

In [130]:
isinstance(df_feedback.loc[5]['user'], np.float64)

True

In [68]:
df_feedback.reset_index(drop=False)[df_feedback.reset_index(drop=False)['user'] == user_id]

,user,item,feedback_value,timestamp
207825,0,12160,3.0,1405209600


In [85]:
len(df_user_temp.shape)

1

In [87]:
items, values, df_user_temp.shape, df_user_temp['feedback_value']

(12160.0, array([1, 1, 1]), (3,), 3.0)

In [56]:
list(items)

TypeError: 'numpy.float64' object is not iterable

In [50]:
dict(zip(items, values))

{2650: 1, 10197: 1}

In [48]:
values

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
%%time
set_items = set(df_feedback['item'].unique())
user_series = np.array([])
first_user = True
count = 1
# for user in model.users:
for user in df_ratings['user'].unique():
    start_time = time.time()
    print ('[{}] {}/{}'.format(start_time, count, len(df_feedback['user'].unique())))
    set_items_seen_user = set(df_feedback[df_feedback['user'] == user]['item'].unique())
    set_items_unobserved = set_items-set_items_seen_user  
    
    df_item_count_temp = df_item_count.loc[set_items_unobserved][:rank_length] 
    print ('[{}] {}/{}'.format(time.time()-start_time, count, len(df_feedback['user'].unique())))        
    user_series = np.append(user_series, np.repeat(user, df_item_count_temp.shape[0]))
    if (first_user):
        df_ranking_temp = df_item_count_temp.copy()
        first_user = False
    else:
        df_ranking_temp = df_ranking_temp.append(df_item_count_temp)
    count += 1   
    
df_ranking_temp['user'] = user_series   
ranking = df_ranking_temp.reset_index(drop = False)[['user', 'item', 'count']].values.tolist()
# del df_item_count_temp
#     ranking.append((user, list(df_item_count_temp.index), list(df_item_count_temp['count'])))
    

In [ ]:
ranking

In [ ]:
df_feedback.loc[~df_feedback.index.isin(df_items_seen_user.item)]

In [ ]:
df_ranking_temp.reset_index(drop = False)[['user', 'item', 'count']].values.tolist()

In [ ]:
ranking[:10]

In [ ]:
set_items-set(df_feedback[df_feedback['user'] == user]['item'].unique())